# Vocabulary Challenge

Se propone resolver el challenge propuesto en clase

In [1]:
!pip install ebooklib
!pip install beautifulsoup4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.5/115.5 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for ebooklib: filename=EbookLib-0.18-py3-none-any.whl size=38778 sha256=6ab846d691b558a8eb45f1586e5e59eccc6e5d237e2b281d15a522a2548963de
  Stored in directory: /root/.cache/pip/wheels/65/2b/63/68307c736d5a2fafeebe9df3e5eccacfe892204ce1fd31a03c
Successfully built ebooklib


In [2]:
import pandas as pd
import re
import string
import unicodedata
from collections import Counter
from google.colab import files
import ebooklib
from ebooklib import epub
from bs4 import BeautifulSoup

## Obtención de datos en csv y limpieza de puntuación y caracteres especiales

Se va a tomar el archivo de los miserables .epub descargado de https://aprende.org/pruebasat?sectionId=6

Este archivo se tiene de manera local y se adjunta al notebook en colab. (Se adjunta el archivo en el directorio de este notebook)

In [3]:
# Subir archivo EPUB
uploaded = files.upload()
epub_filename = list(uploaded.keys())[0]

Saving Los-miserables.epub to Los-miserables.epub


In [4]:
# Extraer texto de EPUB
def extract_text_from_epub(epub_path):
    book = epub.read_epub(epub_path)
    text = []
    for item in book.get_items():
        if item.get_type() == ebooklib.ITEM_DOCUMENT:
            soup = BeautifulSoup(item.get_body_content(), 'html.parser')
            text.append(soup.get_text())
    return '\n'.join(text)

In [5]:
# Eliminar acentos
def remove_accents(text):
    return ''.join(
        c for c in unicodedata.normalize('NFD', text) if unicodedata.category(c) != 'Mn'
    )

# Limpiar texto
def clean_text(text):
    text = text.lower()
    text = remove_accents(text)  # Remover acentos
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)  # Remover caracteres especiales y puntuación
    text = re.sub(r'\s+', ' ', text).strip()  # Remover espacios extras
    return text

In [6]:
# Extraer y limpiar texto
raw_text = extract_text_from_epub(epub_filename)
cleaned_text = clean_text(raw_text)

/usr/local/lib/python3.11/dist-packages/ebooklib/epub.py:1395: UserWarning: In the future version we will turn default option ignore_ncx to True.
  warnings.warn('In the future version we will turn default option ignore_ncx to True.')
/usr/local/lib/python3.11/dist-packages/ebooklib/epub.py:1423: FutureWarning: This search incorrectly ignores the root element, and will be fixed in a future version.  If you rely on the current behaviour, change it to './/xmlns:rootfile[@media-type]'
  for root_file in tree.findall('//xmlns:rootfile[@media-type]', namespaces={'xmlns': NAMESPACES['CONTAINERNS']}):


In [7]:
# Guardar en CSV
df_text = pd.DataFrame({'text': [cleaned_text]})
df_text.to_csv("book_text.csv", index=False)

## Creación del vocabulario

In [8]:
# Crear lista de palabras
words = cleaned_text.split()

In [9]:
# Contar frecuencia de palabras
word_counts = Counter(words)

In [10]:
# Crear DataFrame de vocabulario
df_vocab = pd.DataFrame(word_counts.items(), columns=['word', 'frequency'])
df_vocab = df_vocab.sort_values(by='frequency', ascending=False)

In [11]:
df_vocab

,word,frequency
7,de,5325
22,la,3918
78,que,3818
50,el,3394
16,y,3123
...,...,...
7325,ao,1
804,unidos,1
7318,satanas,1
7317,planes,1


In [12]:
# Guardar en Parquet
df_vocab.to_parquet("vocabulary.parquet", index=False)

## Estadística

In [13]:
# Estadísticas
num_words = len(words)
num_unique_words = len(df_vocab)
top_100 = df_vocab.head(100)
least_100 = df_vocab.tail(100)

print(f"Total de palabras en el texto: {num_words}")
print(f"Palabras únicas en el vocabulario: {num_unique_words}")

Total de palabras en el texto: 109377
Palabras únicas en el vocabulario: 13175


In [14]:
print("Top 100 palabras más frecuentes:")
top_100

Top 100 palabras más frecuentes:


,word,frequency
7,de,5325
22,la,3918
78,que,3818
50,el,3394
16,y,3123
...,...,...
670,voz,111
456,ojos,107
173,alli,107
54,monsenor,105


In [16]:
print("Top 100 palabras menos frecuentes:")
least_100

Top 100 palabras menos frecuentes:


,word,frequency
7367,odry,1
7365,bailaba,1
7364,bigottini,1
7363,pellegrini,1
7362,reves,1
...,...,...
7325,ao,1
804,unidos,1
7318,satanas,1
7317,planes,1


## Extra. Eliminando StopWords

Muchas veces, es conveniente no incluír palabras que suelen repetirse mucho y no aportan información significativa como pueden ser artículos, preposiciones, etc...
Probemos eliminar este tipo de palabras del texto con el fin de explorar una alternativa de vocablario.

In [17]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [18]:
# Obtener lista extensa de stopwords en español
es_stopwords = set(stopwords.words('spanish'))

In [21]:
# Eliminar stopwords
filtered_words = [word for word in words if word not in es_stopwords]
filtered_word_counts = Counter(filtered_words)

df_filtered_vocab = pd.DataFrame(filtered_word_counts.items(), columns=['word', 'frequency'])
df_filtered_vocab = df_filtered_vocab.sort_values(by='frequency', ascending=False)

In [22]:
# Estadísticas después de eliminar stopwords
num_words_filtered = len(filtered_words)
num_unique_words_filtered = len(df_filtered_vocab)
top_100_filtered = df_filtered_vocab.head(100)
least_100_filtered = df_filtered_vocab.tail(100)

print(f"Total de palabras después de eliminar stopwords: {num_words_filtered}")
print(f"Palabras únicas después de eliminar stopwords: {num_unique_words_filtered}")

Total de palabras después de eliminar stopwords: 57301
Palabras únicas después de eliminar stopwords: 13001


In [23]:
print("Top 100 palabras más frecuentes sin stopwords:")
top_100_filtered

Top 100 palabras más frecuentes sin stopwords:


,word,frequency
101,habia,858
67,mas,513
41,senor,447
191,hombre,363
195,si,358
...,...,...
149,paso,69
1603,luz,69
816,cosas,68
346,palabra,66


In [24]:
print("Top 100 palabras menos frecuentes sin stopwords:")
least_100_filtered

Top 100 palabras menos frecuentes sin stopwords:


,word,frequency
7015,ninez,1
7013,alzad,1
7011,turbios,1
7010,claveteado,1
7065,llamando,1
...,...,...
6959,sensaciones,1
6956,huyera,1
6953,perteneceis,1
6952,subrayado,1
